In [18]:
import pandas as pd

In [19]:
weather_data = pd.read_csv('impute/sorted_precipitation_data.csv', dtype={'week': 'str'})

print(weather_data.head())

  ESTACAO    week  precipitation_max  precipitation_avg
0    A601  201201               66.2          14.171429
1    A621  201201               11.2           2.714286
2    A624  201201               17.2           5.771429
3    A603  201201               19.0           9.685714
4    A604  201201               18.4           7.514286


In [20]:
stations_file_path = "impute/stations_RIO.csv"
output_file_path = "impute/sorted_precipitation_data_with_stations.csv"

stations_data = pd.read_csv(stations_file_path)

# Merge the data based on the station ID
merged_data = pd.merge(
    weather_data,
    stations_data[['id_station', 'lat', 'lon', 'lvl']],
    left_on='ESTACAO',
    right_on='id_station',
    how='left'
)

# Drop the now-redundant 'id_station' column
merged_data = merged_data.drop(columns=['id_station'])

# Save the merged data to a new CSV file
merged_data.to_csv(output_file_path, index=False)

print(f"Merged data saved to {output_file_path}")

Merged data saved to impute/sorted_precipitation_data_with_stations.csv


In [21]:
import pandas as pd
from pykrige.uk3d import UniversalKriging3D

def interpolate_precipitation_for_city(file_path, target_lat, target_lon, target_lvl):
    df = pd.read_csv(file_path)
    weeks = df['week'].unique()
    results = []

    for week in weeks:
        week_data = df[df['week'] == week]

        if len(week_data) < 3:
            continue

        x = week_data['lon'].values
        y = week_data['lat'].values
        z = week_data['lvl'].values

        p_max = week_data['precipitation_max'].values
        p_avg = week_data['precipitation_avg'].values

        uk_max = UniversalKriging3D(
            x, y, z, p_max,
            variogram_model='linear',
            drift_terms=['regional_linear']
        )
        interp_max, _ = uk_max.execute(
            'points', [target_lon], [target_lat], [target_lvl]
        )

        uk_avg = UniversalKriging3D(
            x, y, z, p_avg,
            variogram_model='linear',
            drift_terms=['regional_linear']
        )
        interp_avg, _ = uk_avg.execute(
            'points', [target_lon], [target_lat], [target_lvl]
        )

        results.append({
            'week': week,
            'precipitation_max': interp_max[0],
            'precipitation_avg': interp_avg[0]
        })

    return pd.DataFrame(results)


In [22]:
# city_lat = -22.9
# city_lon = -43.2
# city_lvl = 20  # Example altitude
# 
# df_result = interpolate_precipitation_for_city(
#     "impute/sorted_precipitation_data_with_stations.csv",
#     city_lat, city_lon, city_lvl
# )
# 
# print(df_result)


In [23]:
# Load all cities
cities_df = pd.read_csv("impute/filtered_RJ_lat_long_data.csv")

# Store final results
all_results = []

count = 0

# Iterate over each city
for _, row in cities_df.iterrows():
    city = row['CITY']
    state = row['STATE']
    lat = row['LAT']
    lon = row['LONG']
    alt = row['ALT']
    
    count += 1
    
    # Print city information
    print(f"Processing city {count}: {city}, {state}")

    # Get precipitation for this city
    df_result = interpolate_precipitation_for_city(
        "impute/sorted_precipitation_data_with_stations.csv",
        lat, lon, alt
    )

    # Add city and state to each week's row
    df_result['city'] = city
    df_result['state'] = state

    all_results.append(df_result)

# Combine all data
combined_df = pd.concat(all_results, ignore_index=True)

# Save to CSV
combined_df.to_csv("impute/final_city_precipitation.csv", index=False)
print("Final precipitation data for all cities saved.")

Processing city 1: Angra Dos Reis, RJ
Processing city 2: Aperibé, RJ
Processing city 3: Araruama, RJ
Processing city 4: Areal, RJ
Processing city 5: Armação Dos Búzios, RJ
Processing city 6: Arraial Do Cabo, RJ
Processing city 7: Barra Do Piraí, RJ
Processing city 8: Barra Mansa, RJ
Processing city 9: Belford Roxo, RJ
Processing city 10: Bom Jardim, RJ
Processing city 11: Bom Jesus Do Itabapoana, RJ
Processing city 12: Cabo Frio, RJ
Processing city 13: Cachoeiras De Macacu, RJ
Processing city 14: Cambuci, RJ
Processing city 15: Campos Dos Goytacazes, RJ
Processing city 16: Cantagalo, RJ
Processing city 17: Carapebus, RJ
Processing city 18: Cardoso Moreira, RJ
Processing city 19: Carmo, RJ
Processing city 20: Casimiro De Abreu, RJ
Processing city 21: Comendador Levy Gasparian, RJ
Processing city 22: Conceição De Macabu, RJ
Processing city 23: Cordeiro, RJ
Processing city 24: Duas Barras, RJ
Processing city 25: Duque De Caxias, RJ
Processing city 26: Engenheiro Paulo De Frontin, RJ
Proce